# SP02_Clase01c

## k-NN con Scikit-Learn para Regresión

# Problema

Ya que estamos acostumbrados a trabajar con el dataset Iris ... y en él tenemos 4 columnas que son numéricas ... podríamos ver qué tan bien podríamos pronosticar una de ellas en función de las otras tres!

Así que pongámonos como meta, **pronosticar sepal width**, que pasaría a ser el target,  en función de las otras 3 variables numéricas, dejaremos de lado la columna iris.  

Usemos k=5 y 123 para sembrar los valores random.

Como siempre vamos a aplicar nuestro esquema de trabajo:  

- **Datos**
    - Carga de Datos
    - Train y Test Sets
    - Adaptación de los Datos para Scikit-Learn 
        - **X | y** por separado
        - deben ser de dimensión 2 (filas y columnas)
    

- **Modelo**
    - Creación del Modelo
    - Training con fit
    - Parámetros del modelo (si los tiene!)  o visualización en el caso del Árbol.
    

- **Evaluación** del Modelo según corresponda
    - Clasificación:
        - Accuracy
        - F1
        - Confussion Matrix
        - AUC (todavía no lo vimos)
        - Otros (si corresponde)
    - Regresión:
        - $R^2$ Coeficiente de Determinación
        - MSE: Mean Square Error
        - RMSE: Root Mean Square Error
        - Otros (si corresponde)
    

In [1]:
import numpy as np
import pandas as pd

## Datos

El dataset iris.tab lo encontrará en *data/iris.tab*, recordemos que el separador era TAB (/t) y que las dos primeras filas no contenían datos

In [2]:
### Carga de los datos

In [3]:
df_iris=pd.read_csv("data/iris.tab",sep='\t',skiprows=(1,2))
df_iris.head()

,sepal length,sepal width,petal length,petal width,iris
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
df=df_iris[['sepal length','sepal width','petal length', 'petal width']]
df

,sepal length,sepal width,petal length,petal width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


le cambiamos los nombres

In [5]:
df.columns=['sl','sw','pl','pw']

In [6]:
df.head()

,sl,sw,pl,pw
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


### Train y Test Sets

Vamos a separar en 80/20 y para la siembra usaremos 123

In [7]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2,random_state=123)

### Adaptamos los datos para Scikit-Learn

Recordemos que esta vez **el target será sw**

In [8]:
X_train=df_train[['sl','pl','pw']]
y_train=df_train[['sw']]

In [9]:
X_test=df_test[['sl','pl','pw']]
y_test=df_test[['sw']]

### Modelo kNN

Vamos a elegir un valor para la cantidad de vecinos que participarán en la votación, digamos **k=5** que es el valor que utiliza por defecto sklearn.

In [10]:
k=5

Esta vez deberemos utilizar la versión para Regresion de kNN 

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html


In [11]:
from sklearn import neighbors
modelo = neighbors.KNeighborsRegressor(n_neighbors=k,weights='uniform')


### Entrenamos con fit

In [12]:
modelo.fit(X_train, y_train)

KNeighborsRegressor()

### Pronosticamos los resultados para el X_test

In [13]:
y_pred=modelo.predict(X_test)

In [14]:
y_pred[0:10]

array([[2.76],
       [3.14],
       [3.22],
       [2.64],
       [3.12],
       [3.02],
       [2.84],
       [3.  ],
       [3.  ],
       [2.88]])

In [16]:
y_test.head()

,sw
72,2.5
112,3.0
132,2.8
88,3.0
37,3.1


### Evaluación

In [17]:
from sklearn.metrics import mean_squared_error, r2_score
print("EVALUACIÓN del modelo kNN: ")
print("Coeficiente de Determinación R2: ",round(modelo.score(X_test,y_test),3)) # evaluamos el modelo
y_pred = modelo.predict(X_test)
MSE=mean_squared_error(y_test, y_pred)
print("MSE: ",round(MSE,3))
RMSE=mean_squared_error(y_test, y_pred,squared=False)
print("RMSE: ",round(RMSE,3))

EVALUACIÓN del modelo kNN: 
Coeficiente de Determinación R2:  0.478
MSE:  0.084
RMSE:  0.29


El Coeficiente de Determinación no es muy bueno, sin embargo he visto muchos trabajos en producción con valores inferiores a 0.478, no es taaan malo. 

De las medidas de evaluación la única que tiene su valor en las mismas unidades que la variable target es el RMSE (recuerde que MSE es el cuadrado de esta magnitud), así que podríamos comparar el valor del RMSE con el promedio de sw (la variable que estamos pronosticando). Usaremos los valores de sw del Train Set, ya que mientras estemos evaluando no debemos hacer nada que nos permita aprender del test set

In [19]:
sw_promedio=y_train.mean()[0]
sw_promedio

3.040833333333333

Finalmente:

In [20]:
porcentaje=RMSE/sw_promedio * 100
print("El RMSE es el: ", round(porcentaje,2), " % del valor promedio de sw")

El RMSE es el:  9.53  % del valor promedio de sw


 ## Autoevalación 1  

# El Problema:

Vamos a resolver nuevamente el problema de Regresión de las Autoevaluaciones 1 y 2 de la SP1, pero esta vez usaremos kNN.  
Aplique **k=5**, y los mismos valores de **80/20 para Train y Test** y **123** para la generación de números aleatorios.  

- Los datos se encuentran en *data/diabetes_3variables.csv*. 

- Cree el modelo y obtenga los valores de **$R^2$, MSE y RMSE**

- En el archvivo *SP02_Autoeval_01* lo encontrará resuelto. 
